# Introduction

The year 2023 has been marked by economic instability, characterized by high inflation and interest rates, significantly impacting daily life. Given that housing constitutes the largest portion of household expenses, it is evident that these economic factors would exert a substantial influence on the housing market. As we approach the end of 2023, it is opportune to conduct a comprehensive overview of the US housing market, leveraging data from Zillow, a prominent online real estate marketplace that encompasses information on approximately 100 million homes. <br>

This project aims to visualize and conduct a high-level analysis of the US housing market using a dataset comprising around 25,000 listings sourced from Zillow. The dataset encompasses a diverse range of housing characteristics across different states. The initial phase involves data cleaning using Pandas in Python, followed by visualization of the dataset using Power BI. This dual approach allows for a robust examination of trends and patterns within the housing market, providing insights into the potential impact of economic conditions on the real estate landscape.

# Acknowledgement

The dataset utilized in this project has been sourced from the Kaggle dataset titled "United States House Listings: Zillow Extract 2023," generously contributed by Febin Philips. For those interested, the dataset can be accessed at the following Kaggle link: https://www.kaggle.com/datasets/febinphilips/us-house-listings-2023. <br>

Furthermore, it's important to note that the house listings originate from Zillow, a well-known online real estate marketplace. The Zillow platform, available at https://www.zillow.com/, serves as a comprehensive source for real estate data, playing a crucial role in this project's exploration and analysis of the US housing market.

# 1: Import Libraires

In [2]:
import pandas as pd
import numpy as np

# 2: Data Cleaning

## 2.1: Load Dataframe

In [3]:
original_df = pd.read_csv('original_extracted_df.csv')

In [4]:
df = original_df.copy()

In [5]:
df.head()

,State,City,Street,Zipcode,Latitude,Longitude,Bedroom,Bathroom,Area,PPSq,LotArea,ConvertedLot,LotUnit,MarketEstimate,RentEstimate,Price
0,AL,Saraland,Scott Dr,36571.0,30.819534,-88.095960,4.0,2.0,1614.0,148.636927,0.38050,0.38050,acres,240600.0,1599.0,239900.0
1,AL,Southside,Big Oak Dr W,35907.0,33.881050,-86.012436,3.0,2.0,1474.0,0.000678,0.67034,0.67034,acres,186700.0,1381.0,1.0
2,AL,Robertsdale,Cowpen Creek Rd,36567.0,30.590004,-87.580376,3.0,2.0,1800.0,144.388889,3.20000,3.20000,acres,NaN,NaN,259900.0
3,AL,Gulf Shores,Spinnaker Dr #201,36542.0,30.284956,-87.747920,2.0,2.0,1250.0,274.000000,NaN,NaN,NaN,NaN,NaN,342500.0
4,AL,Chelsea,Mallet Way,35043.0,33.357986,-86.608700,3.0,3.0,2224.0,150.629496,0.26000,0.26000,acres,336200.0,1932.0,335000.0


Here are the column description provided by Febin from the Kaggle Datasets:
- State: The state in which the property is located (AL:Alabama) . Includes all US states except Hawaii.
- City: The city where the property is situated.
- Street: The street address of the property.
- Zipcode: The postal code associated with the property.
- Latitude: The latitude coordinates of the property.
- Longitude: The longitude coordinates of the property.
- Bedroom: The number of bedrooms in the house.
- Bathroom: The number of bathrooms in the house.
- Area(sqft): The total area of the house.
- PPSq(Price Per Square Foot): The cost per square foot of the property.
- LotArea(acres or sqft): The total land area associated with the property, in the unit indicated in the column LotUnit.
- ConvertedLot(acres): Converted LotArea with the consistent unit of acres
- LotUnit: The unit used in LotArea.
- MarketEstimate(Dollars \\$): Estimated market value of the property. This value is estimated using Zillow's own algorithm.
- RentEstimate(Dollars \\$): Estimated rental value of the property. This value is estimated using Zillow's own algorithm.
- Price(Dollars \\$): The listed price of the property.

In [6]:
# Drop empty rows (all with nan)
df = df.dropna(axis=0, how='all')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24135 entries, 0 to 24520
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           24135 non-null  object 
 1   City            24135 non-null  object 
 2   Street          24104 non-null  object 
 3   Zipcode         24135 non-null  float64
 4   Latitude        24135 non-null  float64
 5   Longitude       24135 non-null  float64
 6   Bedroom         23456 non-null  float64
 7   Bathroom        23524 non-null  float64
 8   Area            23029 non-null  float64
 9   PPSq            22784 non-null  float64
 10  LotArea         23103 non-null  float64
 11  ConvertedLot    23103 non-null  float64
 12  LotUnit         23103 non-null  object 
 13  MarketEstimate  15927 non-null  float64
 14  RentEstimate    17649 non-null  float64
 15  Price           24130 non-null  float64
dtypes: float64(12), object(4)
memory usage: 3.1+ MB


We can see that except the columns "State", "City", "Zipcode", "Latitude" and "Longitude", other columns contain some missing values, and may need to change or keep depending on the situation, which will be addressed in the following sections.

## 2.2: Cleaning by Column

### 2.2.1: State and Zipcode

Even though it looks like the "State" and "Zipcode" columns are complete, it's a good idea to double-check if all the information makes sense. We want to make sure that the states are listed correctly and that the ZIP codes are in the right numeric format. This check ensures that our dataset has accurate location details, which is important for any comparisons between states later on in the project.

In [8]:
abbr_excel = pd.read_excel('State Abbreviations.xlsx', index_col=1)
abbr_list = abbr_excel.index.values
zipcode_first = abbr_excel.to_dict()['Zipcode first digit']

In [9]:
# Check if State is valid
if set(df['State'].unique()).issubset(set(abbr_list)):
    print('All states in the datasets are valid.')
else:
    print('Some states are invalid.')

All states in the datasets are valid.


In [10]:
# Change Zipcode to str
# Put 0 at the front for 4-digit zipcode
df['Zipcode'] = df['Zipcode'].astype(int).astype(str).apply(lambda x: '0'+x if len(x) == 4 else x)

In [11]:
# Check if zipcode is valid
zipcode_check = df[
    (df['Zipcode'].apply(lambda x: int(x[0])) - df['State'].apply(lambda x: zipcode_first[x]) != 0) &
    ~((df['State'] == 'NY') & (df['Zipcode'].apply(lambda x: x[0]) == '1')) # First digit zipcode for NY can either be 0 (in abbr_excel) or 1
]

if len(zipcode_check) == 0:
    print('All zipcodes belong to their states.')
else:
    print('Some zipcodes do not belong to their states.')

All zipcodes belong to their states.


### 2.2.2: City & Street

In [12]:
df['City'] = df['City'].apply(str.strip)

Some house listings lack information about the street address, which may not be essential for high-level analyses across individual states. However, when considering more detailed intra-state visualizations and comparisons in later stages, having complete "Street" data becomes more relevant. <br> <br>
To improve the dataset's usability for users involved in intra-state assessments, it's recommended to clean and fill in any missing or incomplete entries in the "Street" column. Taking this proactive step ensures that the dataset is ready for future in-depth analyses, allowing users to have a more comprehensive and meaningful exploration of housing data across various states.

In [13]:
# Replace NaN for Street
df.loc[df[df['Street'].isna()].index, 'Street'] = '<No Street Provided>'

In [14]:
df['Street'] = df['Street'].apply(str.strip)

In [15]:
# We consider only the currently available listings
# Remove 'Homes Available Soon' rows
df.drop(df[df['Street'].apply(lambda x: True if 'Homes Available Soon' in x else False)].index, axis=0, inplace=True)

### 2.2.3: Bedroom, Bathroom, Area

In [17]:
df[df['Bedroom'].isna()].head()

,State,City,Street,Zipcode,Latitude,Longitude,Bedroom,Bathroom,Area,PPSq,LotArea,ConvertedLot,LotUnit,MarketEstimate,RentEstimate,Price
374,AL,Ariton,Highway 231 NE,36311,31.581430,-85.77018,NaN,NaN,NaN,NaN,0.90,0.90,acres,NaN,NaN,30000.0
2457,CA,Madera,<No Street Provided>,93636,36.935925,-119.80851,NaN,0.0,NaN,NaN,80.37,80.37,acres,NaN,1500.0,1440000.0
2783,CO,Wiggins,144th Ave,80654,39.960526,-104.13577,NaN,NaN,NaN,NaN,237.75,237.75,acres,NaN,NaN,758000.0
2847,CO,Carbondale,Bald Eagle Way,81623,39.436733,-107.26491,NaN,0.0,3877.0,1050.0,0.50,0.50,acres,NaN,8000.0,4070850.0
2851,CO,Carbondale,Bald Eagle Way,81623,39.437046,-107.26602,NaN,0.0,3867.0,1050.0,0.50,0.50,acres,2532200.0,7899.0,4060350.0


In [18]:
len(df[df['Bedroom'].isna()])

654

It has been observed that a portion of the dataset contains null values in the fields representing the number of bedrooms, bathrooms, and area. Upon closer examination of several listings on Zillow, it became apparent that the reasons for these null values are not consistent. <br> For instance, the 374th listing pertains to a land plot without any constructed house, rendering the mentioned columns irrelevant. On the other hand, the absence of values in the 2847th listing is attributed to a data entry error, where the seller specified the number of bedrooms and bathrooms solely in the description. In rare instances, certain houses indeed lack bedrooms and bathrooms. <br> <br>
Given that the primary goal of this project is to offer an overview of the housing market, and considering that the proportion of missing values is relatively small (around 2%), a pragmatic approach is to designate these values as "No Input." This strategy minimizes the impact on reporting while effectively addressing the varied nature of missing data in the dataset.

### 2.2.4: Lot Area

The columns "LotArea", "ConvertedLot" and "LotUnit" can be combined into one using the same unit, acres.

In [19]:
df.drop(columns=['LotArea', 'LotUnit'], axis=1, inplace=True)
df.rename(columns={"ConvertedLot": "LotAreaAcres"}, inplace=True)

### 2.2.5: Price

House prices are a pivotal feature in our analysis for obtaining a comprehensive overview. Therefore, we are making efforts to estimate prices for missing values and rectify any unreasonable values in this column. This proactive approach aims to enhance the accuracy of our analysis, ensuring that the dataset is as reliable as possible for gaining meaningful insights into the housing market.

In [20]:
# Replace NaN Price with MarketEstimate
target_df = df[(df['Price'].isna()) & (df['MarketEstimate'].notna())]
df.loc[target_df.index, 'Price'] = df.loc[target_df.index, 'MarketEstimate']

# Replace NaN Price and no MarketEstimate with Area * PPSq
target_df = df[(df['Price'].isna()) & (df['Area'].notna()) & (df['PPSq'].notna())]
df.loc[target_df.index, 'Price'] = df.loc[target_df.index, 'Area'] * df.loc[target_df.index, 'PPSq']

# Drop NaN Price
df.drop(df[df['Price'].isna()].index, axis=0, inplace=True)

In [22]:
# Replace unreasonable low Price (0, 1 or lower than RentEstimate) with MarketEstimate
target_df = df[((df['Price'] <= 1) | (df['Price'] <= df['RentEstimate'])) & (df['MarketEstimate'].notna())]
df.loc[target_df.index, 'Price'] = df.loc[target_df.index, 'MarketEstimate']

# Replace unreasonable low Price and no MarketEstimate with Area * PPSq
target_df = df[((df['Price'] <= 1) | (df['Price'] <= df['RentEstimate'])) & (df['MarketEstimate'].isna()) & (df['Area'].notna()) & (df['PPSq'].notna())]
df.loc[target_df.index, 'Price'] = df.loc[target_df.index, 'Area'] * df.loc[target_df.index, 'PPSq']

# Drop unreasonable low Price
target_df = df[(df['Price'] <= 1) | (df['Price'] <= df['RentEstimate'])]
df.drop(target_df.index, axis=0, inplace=True)

## 2.3: Export Cleaned Dataframe

In [23]:
df.to_csv('cleaned_df.csv')

# 3: Data Visualization

Once the dataset is cleaned, we will visualize the data using Power BI. The interactive dashboard is accessible through the provided pbix file, allowing users to explore the data dynamically. Alternatively, the following snapshots offer a glimpse into the dashboard, providing key insights derived from the analysis.

## 3.1: Summary

In total, our dataset comprises 24,070 listings across the United States, with a median house price of \\$335,000. The dashboard employs a United States color map to illustrate housing prices in different states, where a more goldish color indicates a higher price and a more bronze hue signifies a more affordable price. Users can hover over each state to view its specific median house price. <br> <br>
In the top right-hand corners of the dashboard, a succinct summary showcases the top three most expensive and top three most affordable states in the U.S. As of 2023, California, Colorado, and Massachusetts lead in median house prices, ranging from \\$550,000 to \\$805,000. Conversely, West Virginia, Mississippi, and Ohio emerge as the most affordable states, with median prices ranging from \\$190,000 to \\$225,000.

<img src="Snapshots/Main_Median.png"/>

As we delve into the data, it becomes evident that the average (arithmetic mean) of house prices significantly exceeds the median, indicating a right-skewed distribution — a common characteristic in housing price data. To accommodate user preferences, the dashboard provides an option to switch between metrics, allowing users to choose between median and average values. However, it is strongly recommended to use the "Median" metric, as it provides a more accurate reflection of central tendency for right-skewed data. The "Average" metric may be influenced by extreme values. <br> <br>
Under the "Average" mode, the overall average house price for the U.S. increases to \\$532,000. Montana and Nevada join California as the top three states with the highest average prices, while the top three most affordable states remain unchanged.

<img src="Snapshots/Main_Average.png"/>

## 3.2: Filters

The dashboard is enhanced with a set of filters on the left, empowering users to select their preferred features for analysis. The color map dynamically adjusts to display price trends across different states based on the selected features.

For instance, when filtering for a budget of \\$500,000, there are 1,133 available house listings ranging from 1,000 to 1,500 sqft with 1-2 bedrooms and bathrooms across the U.S. The median price for these listings is \\$239,000, providing users with a focused and customized view of the housing market based on their specified criteria.

<img src="Snapshots/Filtered_1.png"/>

Here's another example of using filters in the dashboard.

<img src="Snapshots/Filtered_2.png"/>

## 3.3: Findings

Upon exploration of the dashboard, several key insights on U.S. housing prices in 2023 have emerged from the data: <br>
- The significant difference between the average and median prices suggests a right-skewed distribution in U.S. housing prices. This is common in real estate markets where luxury properties can significantly drive up the average price. <br> <br>
- There is a trend of higher housing prices on the west coast, followed by the east coast, while the central part of the country tends to offer more affordable housing options. The higher prices may be attributed to factors like booming tech industries, urbanization, and desirable living conditions. <br> <br>
- Contrary to expectations, New York, despite being a globally renowned business center, does not boast the highest housing prices compared to other U.S. states. The observation may stem from the fact that New York accommodates a broad range of housing options, not solely consisting of high-end properties. Additionally, factors like neighborhood characteristics and accessibility can contribute to varied housing prices within the state. <br> <br>
- Colorado, situated in the central part of the U.S., exhibits relatively higher housing prices compared to states with a similar geographical location. This could be influenced by its appeal as a destination with a strong economy, outdoor recreational opportunities, and an overall high quality of life. Factors such as population growth, limited housing supply, and increased demand for desirable locations within the state may contribute to the higher prices.

# 4: Further Project Suggestions

These suggestions aim to deepen the insights gained from the current analysis and provide a foundation for more comprehensive and forward-looking investigations into the U.S. housing market. <br>

- Intra-State Visualization and Analysis: Extend the analysis to include intra-state visualization and examination. This would involve exploring the housing market dynamics across various cities within a state, providing a more detailed perspective on regional variations and trends. <br><br>
- Time-Series Analysis with Across-Year Data: Incorporate across-year data to perform time-series analysis. This approach allows for the identification of long-term trends, seasonal patterns, and potential cyclicality in housing prices. Examining changes over time can offer valuable insights into the market's evolution. <br><br>
- Automated Web Scraping for Data Verification: Develop an automated web-scraping system to verify and correct missing or inaccurate values in the dataset. This proactive approach ensures data accuracy and maintains the dataset's reliability, contributing to more robust analyses. <br><br>
- Price Prediction Model: Consider developing a price prediction model to forecast house prices based on given features. Machine learning models, such as regression algorithms, can be trained on historical data to predict future housing prices. This predictive model would be a valuable tool for both buyers and sellers in making informed decisions.

Please let me know if you have other suggestion as well. Thank you!